In [1]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk

In [2]:
#toast = pd.read_pickle('../data/ICWSM19_data/disguisedtoasths.pkl')
#dafran = pd.read_pickle('../data/ICWSM19_data/dafran.pkl')
kingrichard = pd.read_pickle('../data/ICWSM19_data/kingrichard.pkl')
scarra = pd.read_pickle('../data/ICWSM19_data/scarra.pkl')
#tsm_hamlinz = pd.read_pickle('../data/ICWSM19_data/tsm_hamlinz.pkl')
#tsm_myth = pd.read_pickle('../data/ICWSM19_data/tsm_myth.pkl')
xchocobars = pd.read_pickle('../data/ICWSM19_data/xchocobars.pkl')
xqcow = pd.read_pickle('../data/ICWSM19_data/xqcow.pkl')

# KR smaller videos

In [3]:
kingrichard.video_id.unique()

array(['276926280', '276511828', '276112720', '275726764', '275316847',
       '274592109', '274216531', '273616046', '271924516', '270929349',
       '270674078', '270350156', '270093931', '269938943', '269675573',
       '269543679', '267613464', '267229722', '266838054', '266439587',
       '266032978', '265418627', '265226178', '264855335', '264485130',
       '264114203', '263344861', '262786607'], dtype=object)

In [5]:
test1 = kingrichard[kingrichard['video_id'] == '264485130']
len(test1)

12047

In [6]:
test2 = kingrichard[kingrichard['video_id'] == '269543679']
len(test2)

7191

In [7]:
test3 = kingrichard[kingrichard['video_id'] == '270350156']
len(test3)

8973

In [8]:
kr_short = pd.concat([test1, test2, test3], axis = 0, ignore_index = True)

In [9]:
len(kr_short)

28211

# SC smaller

In [10]:
scarra.video_id.unique()

array(['276871390', '265145608', '276126026', '265494216', '273665972',
       '256000010', '275591372', '273290281', '270608898', '275948338',
       '277232102', '262534145', '263682474', '274775304', '266981878',
       '267821017', '272206088', '270162155', '267422890', '272522549',
       '262866347', '264726993', '263229758', '269054231', '269855583',
       '264362512', '265885663', '267887160', '268199587', '269400478',
       '270966857', '268659666', '276347730', '260269647', '275181618',
       '264001539'], dtype=object)

In [13]:
test1 = scarra[scarra['video_id'] == '265494216']
len(test1)

11791

In [14]:
test2 = scarra[scarra['video_id'] == '262866347']
len(test2)

16300

In [15]:
sc_short = pd.concat([test1, test2], axis = 0, ignore_index = True)
len(sc_short)

28091

# XCB smaller

In [19]:
xchocobars.video_id.unique()

array(['273695352', '269063618', '260714060', '272928217', '263653805',
       '269427254', '270210427', '272554225', '275596553', '268663172',
       '276776971', '274463688', '265544787', '267441261', '275994445',
       '267883435', '268411278', '260313080', '265914709', '266324109',
       '265140785', '274073677', '272307298', '274938182', '277180226',
       '263260087', '262529008', '273313464'], dtype=object)

In [27]:
test1 = xchocobars[xchocobars['video_id'] == '275994445']
len(test1)

14770

In [36]:
test2 = xchocobars[xchocobars['video_id'] == '274073677']
len(test2)

12158

In [37]:
cb_short = pd.concat([test1, test2], axis = 0, ignore_index = True)
len(cb_short)

26928

# xQc smaller

In [38]:
xqcow.video_id.unique()

array(['277097733', '276435378', '275892945', '274534777', '274357758',
       '274126179', '273749460', '273353339', '272983188', '272971464',
       '271731098', '270484445', '270298976', '269840141', '269367694',
       '268930808', '268737446', '268173015', '267979155', '267439886',
       '266949532', '266807790', '265041320', '264862228', '264272646',
       '263960192', '263718552', '263320813', '262825910', '262437389',
       '260693531', '260322780', '253780433'], dtype=object)

In [42]:
test1 = xqcow[xqcow['video_id'] == '272971464']
len(test1)

14900

In [58]:
test2 = xqcow[xqcow['video_id'] == '273749460']
len(test2)

10468

In [59]:
xq_short = pd.concat([test1, test2], axis = 0, ignore_index = True)
len(xq_short)

25368

# Making final df

In [60]:
dfs = [kr_short, sc_short, cb_short, xq_short]
final_df = pd.concat(dfs, axis = 0, ignore_index = True)

- Drop commenter type as only one value left in it (Got rid of admins/staff)
- Get rid of any edited chats (TO DO) then drop updated_at
    - OR drop updated_at but make new replacement column that marks if edited
- Use fragments to replace text so only text and break out emoticons to seperate column

In [62]:
final_df.columns

Index(['body', 'channel_id', 'commenter_id', 'commenter_type', 'created_at',
       'fragments', 'offset', 'updated_at', 'video_id'],
      dtype='object')

Find only users and drop admin/staff comments

In [63]:
df = final_df[final_df.commenter_type == 'user']

In [64]:
df = df.drop('commenter_type', axis = 1)

Drop any edited Twitch chats (can't tell if commentor or mod edited)

In [65]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['updated_at'] = pd.to_datetime(df['updated_at'])

In [66]:
df['edited'] = df.apply(lambda row: row.updated_at - row.created_at, axis=1)

In [67]:
no_edits = df[df['edited'] == pd.Timedelta("0 days 00:00:00")]

Replace text with non-emoticon one

In [68]:
test = no_edits.iloc[2355]
test.fragments

[{'emoticon_id': '115846'},
 {'text': ' '},
 {'emoticon_id': '115846'},
 {'text': ' '},
 {'emoticon_id': '115846'}]

In [69]:
for d in test.fragments:
    if 'text' in d.keys():
        items = list(d.items())
        print(items[0][1])

In [70]:
def replace_text(fragments):
    twitch_chat = ''
    for d in fragments:
        if 'text' in d.keys():
            if len(twitch_chat) == 0:
                items = list(d.items())
                twitch_chat = items[0][1]
            else:
                items = list(d.items())
                twitch_chat = twitch_chat + ' ' + items[0][1]
    return twitch_chat

In [71]:
no_edits['twitch_chat'] = no_edits['fragments'].apply(replace_text)

Break emoticons out into a list in their own column

In [72]:
def emoticon_list(fragments):
    emotes = []
    for d in fragments:
        if 'emoticon_id' in d.keys():
            items = list(d.items())
            emotes.append(items[0][1])
    if emotes == []:
        return ['None']
    return emotes

In [73]:
no_edits['emotes'] = no_edits['fragments'].apply(emoticon_list)

Get rid of any rows that are chat commands (start w/ !)

In [74]:
test = no_edits.iloc[344]
test.twitch_chat

'START IT'

In [75]:
command = no_edits['twitch_chat'].str.startswith('!')  
no_edits = no_edits[~command].reset_index(drop=True)

In [76]:
no_edits = no_edits.drop(['body', 'fragments', 'updated_at', 'edited'], axis = 1)

Check for duplicates and nulls

In [77]:
no_edits.isna().sum()

channel_id      0
commenter_id    0
created_at      0
offset          0
video_id        0
twitch_chat     0
emotes          0
dtype: int64

In [78]:
um = no_edits.copy()

In [79]:
um = um.drop('emotes', axis = 1)

In [80]:
um.duplicated().sum()

0

In [81]:
no_edits.columns

Index(['channel_id', 'commenter_id', 'created_at', 'offset', 'video_id',
       'twitch_chat', 'emotes'],
      dtype='object')

In [82]:
no_edits

,channel_id,commenter_id,created_at,offset,video_id,twitch_chat,emotes
0,66691674,147879911,2018-05-23 03:08:19.137000+00:00,10.337,264485130,wazupp,[None]
1,66691674,224598352,2018-05-23 03:08:22.299000+00:00,13.499,264485130,Yo,[None]
2,66691674,184337096,2018-05-23 03:08:24.780000+00:00,15.980,264485130,What up,[None]
3,66691674,182777162,2018-05-23 03:08:30.072000+00:00,21.272,264485130,yes!!!,[None]
4,66691674,225109124,2018-05-23 03:08:32.427000+00:00,23.627,264485130,Wassgud,[None]
...,...,...,...,...,...,...,...
106254,71092938,30650660,2018-06-16 02:23:50.439000+00:00,4472.739,273749460,F,[None]
106255,71092938,150039008,2018-06-16 02:23:50.487000+00:00,4472.787,273749460,f,[None]
106256,71092938,180514940,2018-06-16 02:23:50.523000+00:00,4472.823,273749460,f,[None]
106257,71092938,43836580,2018-06-16 02:23:50.537000+00:00,4472.837,273749460,F,[None]


In [83]:
no_edits.to_csv('../data/small_merged_chats', index = False)